# Chapter 5: Tool Use

## Hands-On Code Example (LangChain)

> Adapted and modified from https://docs.google.com/document/d/1bE4iMljhppqGY1p48gQWtZvk6MfRuJRCiba1yRykGNE/edit?tab=t.0
> 
> Do  2 Okt 2025 16:00:42 BST

The implementation of tool use within the LangChain framework is a two-stage process. Initially, one or more tools are defined, typically by encapsulating existing Python functions or other runnable components. Subsequently, these tools are bound to a language model, thereby granting the model the capability to generate a structured tool-use request when it determines that an external function call is required to fulfill a user's query.

The following implementation will demonstrate this principle by first defining a simple function to simulate an information retrieval tool. Following this, an agent will be constructed and configured to leverage this tool in response to user input. The execution of this example requires the installation of the core LangChain libraries and a model-specific provider package. Furthermore, proper authentication with the selected language model service, typically via an API key configured in the local environment, is a necessary prerequisite. 

In [1]:
import os, getpass
import asyncio
import nest_asyncio
from typing import List
from dotenv import load_dotenv
import logging

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [ ]:
# UNCOMMENT
# Prompt the user securely and set API keys as an environment variables
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

try:
  # A model with function/tool calling capabilities is required.
  llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
  print(f"✅ Language model initialized: {llm.model}")
except Exception as e:
  print(f"🛑 Error initializing language model: {e}")
  llm = None

✅ Language model initialized: models/gemini-2.0-flash


E0000 00:00:1759418143.151321  672654 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


## Note
Alternatively, you can load environment variables from a .env file if present with: 

```python
load_dotenv()  
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
``` 

In [3]:
# --- Define a Tool ---
@langchain_tool
def search_information(query: str) -> str:
  """
  Provides factual information on a given topic. Use this tool to find answers to phrases
  like 'capital of France' or 'weather in London?'.
  """
  print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")
  # Simulate a search tool with a dictionary of predefined results.
  simulated_results = {
      "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.",
      "capital of france": "The capital of France is Paris.",
      "population of earth": "The estimated population of Earth is around 8 billion people.",
      "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
      "default": f"Simulated search result for '{query}': No specific information found, but the topic seems interesting."
  }
  result = simulated_results.get(query.lower(), simulated_results["default"])
  print(f"--- TOOL RESULT: {result} ---")
  return result

tools = [search_information]

In [4]:
# --- Create a Tool-Calling Agent ---
if llm:
  # This prompt template requires an `agent_scratchpad` placeholder for the agent's internal steps.
  agent_prompt = ChatPromptTemplate.from_messages([
      ("system", "You are a helpful assistant."),
      ("human", "{input}"),
      ("placeholder", "{agent_scratchpad}"),
  ])

  # Create the agent, binding the LLM, tools, and prompt together.
  agent = create_tool_calling_agent(llm, tools, agent_prompt)

  # AgentExecutor is the runtime that invokes the agent and executes the chosen tools.
  # The 'tools' argument is not needed here as they are already bound to the agent.
  agent_executor = AgentExecutor(agent=agent, verbose=True, tools=tools)

## Note
When your agent runs, the {agent_scratchpad} might look like:

```
I need to search for information about the capital of France.

Action: search_information
Action Input: {"query": "capital of France"}
Observation: The capital of France is Paris.

Now I have the information needed to answer the user's question.
```

The {agent_scratchpad} starts empty for each new input/query. 

In [5]:
async def run_agent_with_tool(query: str):
  """Invokes the agent executor with a query and prints the final response."""
  print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
  try:
      response = await agent_executor.ainvoke({"input": query})
      print("\n--- ✅ Final Agent Response ---")
      print(response["output"])
  except Exception as e:
      print(f"\n🛑 An error occurred during agent execution: {e}")

In [6]:
async def main():
  """Runs all agent queries concurrently."""
  tasks = [
      run_agent_with_tool("What is the capital of France?"),
      run_agent_with_tool("What's the weather like in London?"),
      run_agent_with_tool("Tell me something about dogs.") # Should trigger the default tool response
  ]
  await asyncio.gather(*tasks)

In [7]:
nest_asyncio.apply()
asyncio.run(main())

E0000 00:00:1759418143.209016  672654 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'dogs'}`



--- 🛠️ Tool Called: search_information with query: 'dogs' ---
--- TOOL RESULT: Simulated search result for 'dogs': No specific information found, but the topic seems interesting. ---
Simulated search result for 'dogs': No specific information found, but the topic seems interesting.
Invoking: `search_information` with `{'query': 'weather in London?'}`



--- 🛠️ Tool Called: search_information with query: 'weather in London?' ---
--- TOOL RESULT: Simulated search result for 'weather in London?': No specific information found, but the topic seems interesting. ---
Simulated search result for 'weather in 

## Note
In modern Jupyter, you can also use top-level await: 

In [ ]:
await main()  # Now this works in Jupyter!


--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'dogs'}`



--- 🛠️ Tool Called: search_information with query: 'dogs' ---
--- TOOL RESULT: Simulated search result for 'dogs': No specific information found, but the topic seems interesting. ---
Simulated search result for 'dogs': No specific information found, but the topic seems interesting.
Invoking: `search_information` with `{'query': 'weather in London?'}`



--- 🛠️ Tool Called: search_information with query: 'weather in London?' ---
--- TOOL RESULT: Simulated search result for 'weather in London?': No specific information found, but the topic seems interesting. ---
Simulated search result for 'weather in 

## Note
The agent finds an answer for 'What is the capital of France?', but no answer for 'What's the weather like in London?' and 'Tell me something about dogs.'. That might be a bit odd at first. 

What actually happened underneath is that the LLM rephrased the queries before invoking the tool. We can see that in the output lines: 

``` 
--- 🛠️ Tool Called: search_information with query: 'capital of France' ---
--- 🛠️ Tool Called: search_information with query: 'weather in London?' ---
--- 🛠️ Tool Called: search_information with query: 'dogs' ---
```

These are the actual queries, that are passed into the (function) tool `search_information`. And because the tool only modifies the queries to lowercase and then performs a strict matching dictionary search, we only get a correct answer for 'capital of france'. We don't get the correct answer (but the default answer) for 'weather in London?' because of the question mark at the end. And for 'dogs' we just get the default answer. 

Further, the LLM then modifies the answers from the tool to give a final answer. Here, an overview of what happened: 

| Original Query | LLM Re-phrased Query | lowercase | Tool Answer | LLM Re-phrased Answer |
|---|---|---|---|---|
| "What is the capital of France?" | 'capital of France' | 'capital of france' | "The capital of France is Paris." | The capital of France is Paris. |
| "What's the weather like in London?" | 'weather in London?' | 'weather in london?' | "No specific information found, but the topic seems interesting." | I'm sorry, I don't have the current weather information for London. |
| "Tell me something about dogs."| 'dogs' | 'dogs' | "No specific information found, but the topic seems interesting." | I'm sorry, I don't have any specific information about dogs. But they are interesting! Perhaps you could ask me a more specific question? | 

Feel free, to play around with the `search_information` code, e.g. adding 

```python 
query = query.lower().strip().rstrip('?')
```

or an new dictionary entry for "dogs". 